# Hello World API with Flask

In [3]:
# We'll create the script - we created our folder structure with cookiecutter 
import os
hello_world_script_file = os.path.join(os.path.pardir,'src','models','hello_world_api2.py')

In [13]:
%%writefile $hello_world_script_file

from flask import Flask, request
app = Flask(__name__)

@app.route('/api', methods=['POST']) #api will take an input, process it, and return it

def say_hello():
    data = request.get_json(force=True) #we will pass json, so use get_json to get extract the data
    name = data['name']
    return "hello {0}".format(name)

if __name__ == '__main__': # script entry point, the flask app will run on port 10001 - can be any available port
    app.run(port=10001, debug=True,use_reloader=False) # debug = true for troubleshooting in dev

Overwriting ../src/models/hello_world_api2.py


In [6]:
# We have started the process via the command line with python3 hellow_world_api2.py
import json

In [7]:
import requests

In [14]:
# create a call to the API endpoint
url = 'http://127.0.0.1:10001/api'
#create the data we are sending
data = json.dumps({'name':'graeme'}) #dumps creates the data in a json object
r = requests.post(url, data) #call the API and store response in r.

In [15]:
print(r.text)

hello graeme


In [16]:
# This is calling the API and returning correctly :D